In [1]:
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
gc = gspread.service_account()

In [2]:
scopes = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
          "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file('secrets/linkml-336707-1c85304108ea.json', scopes=scopes)
client = gspread.authorize(creds)


In [3]:
import time

In [4]:
names = [x.strip() for x in open("definitions-sheets/evaluator-ids.txt").readlines()]

In [5]:
dfs = []
all_rows = []
for name in names:
    #print(name)
    wks = gc.open(str(f"Definition Evaluation Template - {name}")).worksheet("Definitions (EDIT HERE)")
    rows = wks.get_all_records()
    rows = [row for row in rows if row["score"] or row["accuracy"]]
    propagated = 0
    for row in rows:
        row["evaluator"] = name
        for k in ["accuracy", "internal_consistency", "score", "confidence"]:
            v = row[k]
            if v == "?":
                v = ""
            if isinstance(v,float):
                v = round(v)
            if isinstance(v,str) and "-" in v:
                # some customized their form
                v = int(v.split("-")[0])
            if v and v not in list(range(1,6)):
                print(f"BAD VALUE {v} in {row['definition']}")
                row[k] = None
        if not row["score"] and row["accuracy"]:
            propagated += 1
            row["score"] = row["accuracy"]
    all_rows.extend(rows)
    wks_df = pd.DataFrame(rows)
    #print("rows", len(rows))
    #print("propagated", propagated)
    wks_df["evaluator"] = name
    time.sleep(3)
    
    dfs.append(wks_df)
#eval_df = pd.concat(dfs)
eval_df = pd.DataFrame(all_rows)

#eval_df["evaluator"].unique()

BAD VALUE   in A structural abnormality in which the sperm neck is bent or curved.


In [6]:
#eval_df.groupby(["ontology", "evaluator"]).size()
eval_df.groupby(["ontology"]).size()

ontology
cl        1917
envo      1151
foodon    1223
go         917
hp         355
mondo      472
mp         265
oba        236
uberon     471
dtype: int64

In [7]:
eval_df = eval_df.rename(columns={"label_x": "label", "ontology_x": "ontology", \
                        "accuracy_x": "accuracy", "score_x": "score", "internal_consistency_x": "consistency", \
                        "confidence_x": "confidence", "notes_x": "notes"})

In [8]:
for k in ["score", "accuracy", "internal_consistency"]:
    eval_df[k] = pd.to_numeric(eval_df[k], errors='coerce')

In [9]:
import hashlib
eval_df['evaluator'] = eval_df['evaluator'].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())

## Save as CSV for later analysis

This is analyzed in a separate notebook

In [10]:
eval_df.to_csv("definitions-sheets/combined.csv", index=False)